In [4]:
from dotenv import load_dotenv

load_dotenv(".env.local")

True

In [7]:
import os

pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [98]:
from langchain.chains import RetrievalQA  
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    model_name='gpt-3.5-turbo',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [99]:
query1 = """What experiences do you have with databases?"""

query2 = """Describe some projects you've worked on that involve machine learning?"""\

print("Query 1\n")
print("Chat with knowledge:")
print(qa.invoke(query1).get("result"))

print("\nQuery 2\n")
print("Chat with knowledge:")
print(qa.invoke(query2).get("result"))


Query 1

Chat with knowledge:
I am an AI assistant and do not have personal experiences.

Query 2

Chat with knowledge:
I don't have the ability to work on projects or have personal experiences.


## Embedding nodes into the PineconeDB

In [2]:
from dotenv import load_dotenv

load_dotenv(".env.local")

True

In [4]:
from neo4j import GraphDatabase
import os

uri = os.getenv("NEO4J_URL")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
driver = GraphDatabase.driver(uri, auth=(username, password))
driver

In [6]:
from langchain_openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

pinecone_api_key = os.getenv("PINECONE_API_KEY")

index_name = "avd-bot"
pc = Pinecone(api_key=pinecone_api_key)

model_name = "text-embedding-3-small"
embeddings = OpenAIEmbeddings(
  model=model_name,
)
index = pc.Index(index_name)
vector_store = PineconeVectorStore(index=index, embedding=embeddings)


In [8]:
node_labels = ["Experience", "Descriptions", "Skills"]

def get_nodes(driver, label):
    with driver.session() as session:
        result = session.run(
            f"MATCH (n:{label}) RETURN n"
        )
        return [record["n"] for record in result]
    
for label in node_labels:
    nodes = get_nodes(driver, label)
    metadata = []
    texts = []

    for node in nodes:
        namespace = label
        for key in node.keys():
            if key == "id":
                continue
            metadata.append({
                "element_id": node.element_id,
                "label": label, 
                "property": key,
                "value": node[key]
            })
            texts.append(node[key])
    
    vector_store.add_texts(texts, metadatas=metadata)




In [135]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    model_name='gpt-4o',
    temperature=0.0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever()
)

In [136]:
results = vector_store.similarity_search_with_score("Do you have experience with databases?", namespace="Skills")
max_score = max([result[1] for result in results])
top_nodes = [result[0] for result in results if result[1] == max_score]
top_nodes

[Document(id='1df215f4-3f4f-421c-a773-74e943f8baf9', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:37', 'label': 'Skills', 'property': 'type', 'value': 'Database'}, page_content='Database'),
 Document(id='0bcb11e8-a160-484e-9569-5250c24edadb', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:35', 'label': 'Skills', 'property': 'type', 'value': 'Database'}, page_content='Database'),
 Document(id='fe51bc3c-b547-465c-9077-e8dd2c1369cb', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:36', 'label': 'Skills', 'property': 'type', 'value': 'Database'}, page_content='Database')]

In [137]:
vector_store.similarity_search_with_score("What did you learn in your masters?", namespace="Experience")

[(Document(id='246433ef-6b83-4ab0-acc4-ab3b7c52de05', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:2', 'label': 'Experience', 'property': 'subtitle', 'value': "Master's in Data Science"}, page_content="Master's in Data Science"),
  0.845497549),
 (Document(id='4d2957d8-5ed6-4ea1-bfbe-a253a7be13f1', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:5', 'label': 'Experience', 'property': 'subtitle', 'value': 'Bacheclor of Science in Computer Engineering'}, page_content='Bacheclor of Science in Computer Engineering'),
  0.809191346),
 (Document(id='828839cf-c318-451e-a80e-eb600a821d55', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be0-d7c69fee4e6d:1', 'label': 'Experience', 'property': 'subtitle', 'value': 'Backend Engineer (Machine Learning and App Development)'}, page_content='Backend Engineer (Machine Learning and App Development)'),
  0.799822688),
 (Document(id='1ba6aa71-d2de-4780-ba81-c44bdd683ede', metadata={'element_id': '4:9eaf6da1-f0ca-4609-9be

In [138]:
qa.invoke("What did you learn in your masters?").get("result")

"I don't have personal experiences or the ability to pursue a master's degree. However, I can provide information on what a typical master's program might cover in various fields if that would be helpful."